In [ ]:
# !conda activate bento
# !pip install bentoml

## Case 1

In [ ]:
import tensorflow as tf
from bentoml import BentoService, api, artifacts
from bentoml.adapters import JsonInput
from bentoml.frameworks.tensorflow import TensorflowSavedModelArtifact

@artifacts([TensorflowSavedModelArtifact('tts_model')])
class TTSService(BentoService):

    @api(input=JsonInput(), batch=False)
    def predict(self, json_input):
        # Load the input data and perform inference with the loaded model
        input_data = json_input['input']
        model = self.artifacts.tts_model
        output = model.inference(input_data)

        return output


## Case 2

- TensorFlowTTS를 BentoML과 함께 사용할 수 있습니다. 
- TensorFlowTTS가 설치된 Docker 컨테이너를 이미 생성했으므로 이전처럼 Dockerfile을 사용하여 컨테이너를 BentoML 서비스로 패키징한 다음 Kubernetes 또는 AWS Elastic Beanstalk와 같은 컨테이너 기반 배포 플랫폼에 배포할 수 있습니다.
- Docker 컨테이너를 BentoML 서비스로 패키징하려면 BentoService 클래스와 TensorflowSavedModelArtifact를 사용하여 저장된 TensorFlowTTS 모델을 로드할 수 있습니다. 다음은 예제 코드입니다.

In [ ]:
import tensorflow as tf
from bentoml import BentoService, api, artifacts
from bentoml.adapters import JsonInput
from bentoml.frameworks.tensorflow import TensorflowSavedModelArtifact
from tensorflow_tts.inference import TFAutoModel

@artifacts([TensorflowSavedModelArtifact('tts_model')])
class TTSService(BentoService):

    def __init__(self):
        self.tts_model = None

    def setup(self):
        self.tts_model = TFAutoModel.from_pretrained(self.artifacts.tts_model)

    @api(input=JsonInput(), batch=False)
    def predict(self, json_input):
        # Load the input data and perform inference with the loaded model
        input_data = json_input['input']
        output = self.tts_model(input_data, training=False)[0]

        return output.numpy()


- 이 예제 코드에서는 TensorFlowTTS.inference의 TFAutoModel을 사용하여 저장된 모델을 로드합니다. 
- 그런 다음 JSON 입력을 받고 로드된 모델로 추론을 수행하는 예측 메서드를 정의합니다. 예측 방법의 출력은 numpy 배열 형식으로 생성된 음성 파형입니다.

- 그런 다음 새 BentoML 서비스 인스턴스를 만들고 다음을 실행하여 디렉터리에 저장할 수 있습니다.

In [ ]:
tts_service = TTSService()
tts_service.pack('tts_model', '/path/to/saved_model')
saved_path = tts_service.save()


- 이렇게 하면 TensorFlowTTS 모델을 아티팩트로 포함하는 saved_path에 BentoML 서비스 번들이 생성됩니다. 

- 그런 다음 저장된 BentoML 서비스를 로드하고 다음과 같이 예측 메서드를 호출할 수 있습니다.

In [ ]:
from bentoml import load

loaded_service = load(saved_path)
output = loaded_service.predict({'input': input_data})


## Case 3

1. Bentoml에 필요한 패키지를 가져오고 TTS.py 파일을 수정하여 Bentoml 서비스를 정의합니다.

In [ ]:
import soundfile as sf
import yaml
import tensorflow as tf
from bentoml import BentoService, api, artifacts
from bentoml.adapters import JsonInput
from bentoml.frameworks.tensorflow import TensorflowSavedModelArtifact
from tensorflow_tts.inference import AutoProcessor
from tensorflow_tts.inference import TFAutoModel


@artifacts([TensorflowSavedModelArtifact('tts_model')])
class TTSService(BentoService):
    
    def preprocess(self, text):
        processor = AutoProcessor.from_pretrained("tensorspeech/tts-tacotron2-kss-ko")
        input_ids = processor.text_to_sequence(text)
        return input_ids

    def predict(self, text):
        # Load the input data and perform inference with the loaded model
        input_ids = self.preprocess(text)
        tacotron2 = self.artifacts.tts_model['tacotron2']
        mb_melgan = self.artifacts.tts_model['mb_melgan']
        decoder_output, mel_outputs, stop_token_prediction, alignment_history = tacotron2.inference(input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),input_lengths=tf.convert_to_tensor([len(input_ids)], tf.int32),speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),)
        audio = mb_melgan.inference(mel_outputs)[0, :, 0]
        # save to file
        sf.write('./audio.wav', audio, 22050, "PCM_16")
        return './audio.wav'


2. Dockerfile에서 다음 줄을 추가하여 bentoml 패키지를 설치합니다.

In [ ]:
RUN pip install bentoml

3. Bentoml을 사용하도록 Entrypoint 을 수정합니다.

In [ ]:
ENTRYPOINT ["bentoml", "serve-gunicorn", "/bento"]

4. Docker image 를 Build 합니다. :

In [ ]:
sudo docker build -t tts-service .

5. save 커맨드를 사용해서 당신의 Bentoml service 를 저장합니다:

In [ ]:
from bentoml import save

service = TTSService()
service.pack('tts_model', {'tacotron2': tacotron2, 'mb_melgan': mb_melgan})
save(service, 'bento')

6. Docker run 커맨드를 사용해서 container 을 build 합니다:

In [ ]:
sudo docker run -p 5000:5000 -v /path/to/bento:/bento tts-service

7. curl 또는 HTTP 클라이언트를 사용하여 API에 요청합니다.

In [ ]:
curl -i \
    --header "Content-Type: application/json" \
    --request POST \
    --data '{"text": "550호 아저씨 95000원 내세요."}' \
    http://localhost:5000/predict

curl은 명령줄에서 사용할 수 있는 HTTP client이며, 다양한 프로토콜을 지원하며, 데이터 전송을 위해 다양한 기능을 제공합니다. HTTP client는 서버에 HTTP 요청을 보내고, 서버가 응답한 데이터를 받아오는 도구입니다.

예를 들어, curl을 사용하여 Google의 메인 페이지를 가져올 수 있습니다. 다음과 같은 명령어를 사용합니다.

curl https://www.google.com


이 명령어는 HTTPS를 사용하여 https://www.google.com 주소로 GET 요청을 보내고, 서버로부터 받은 응답 데이터를 출력합니다.

HTTP client를 사용하면 서버에 요청을 보내고, 서버가 응답한 데이터를 받아와서 처리할 수 있습니다. Python에서는 requests와 urllib 같은 라이브러리를 사용하여 HTTP client를 구현할 수 있습니다. 이 라이브러리들은 URL을 열고 데이터를 읽어오는 간단한 인터페이스를 제공합니다.

예를 들어, requests 라이브러리를 사용하여 Google의 메인 페이지를 가져오는 코드는 다음과 같습니다.

import requests

response = requests.get('https://www.google.com')
print(response.content)


이 코드는 requests.get() 함수를 사용하여 HTTPS를 사용하여 https://www.google.com 주소로 GET 요청을 보내고, 서버로부터 받은 응답 데이터를 출력합니다.